In [ ]:
"""
- LA_TRANSITION_ECOLOGIQUE -> il faut shift les 3 dernières colonnes 



"""

In [3]:
COLS_TO_REMOVE_GDN = ['authorId', 'authorType', 'authorZipCode', 'createdAt', 'id', 'publishedAt', 'reference', 'title', 'trashed', 'trashedStatus', 'updatedAt']
COLS_TO_REMOVE_GDN = ['authorId', 'authorType', 'authorZipCode', 'createdAt', 'id', 'publishedAt', 'reference', 'title', 'trashed', 'trashedStatus', 'updatedAt', 'Tomerge1','Tomerge2']

In [31]:
import pandas as pd
import numpy as np
import spacy
from nltk.corpus import stopwords
import re
spacy_nlp = spacy.load('fr_core_news_sm')
import gensim, spacy
import gensim.corpora as corpora
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [37]:
df = pd.read_csv("data/GDN/DEMOCRATIE_ET_CITOYENNETE.csv", low_memory=False, nrows=20) # 0 à 201907 (rows)

In [52]:
z = list(df)
print(len(z))
print(z[-11:])
# for e in z:
#     print(e)
#     print('-'*10)

122
['authorId', 'authorType', 'authorZipCode', 'createdAt', 'id', 'publishedAt', 'reference', 'title', 'trashed', 'trashedStatus', 'updatedAt']


In [53]:
df = pd.read_csv("data/GDN/LA_TRANSITION_ECOLOGIQUE.csv", low_memory=False, nrows=5000)
df = df.drop(COLS_TO_REMOVE_GDN, axis=1)

In [54]:
cols_name = list(df)[1:] # on retire la cols "id"
n_cols = len(cols_name)
cols_data = [None] * n_cols
for i in range(n_cols):
    col_i = df[cols_name[i]]
    cols_data[i] = col_i[~pd.isnull(col_i)]
data = [i.tolist() for i in cols_data]

In [55]:
emptycols = []
maxtext = 0
maxtext_ind = 0
for i in range(len(data)):
    if not len(data[i]):
        emptycols.append(i)
    else:
#         print(data[i])
#         print(maxtext, (" ".join(data[i])))
        if maxtext < len(" ".join(data[i])):
            maxtext_ind = i
            maxtext = len(" ".join(data[i]))
#             print(len(" ".join(data[i])), i)
print(maxtext, maxtext_ind)
# print(emptycols)

1768921 37


In [56]:
data_c = data[maxtext_ind]
len(data_c), len(" ".join(data_c))

(4469, 1768921)

In [57]:
data_c = [re.sub('\s+', ' ', sent) for sent in data_c]

<input>:1: DeprecationWarning: invalid escape sequence \s
<input>:1: DeprecationWarning: invalid escape sequence \s
<input>:1: DeprecationWarning: invalid escape sequence \s
<ipython-input-57-3c833f97c401>:1: DeprecationWarning: invalid escape sequence \s
  data_c = [re.sub('\s+', ' ', sent) for sent in data_c]


In [58]:
spacy_stopwords = spacy.lang.fr.stop_words.STOP_WORDS
spacy_stopwords.add("€")

In [42]:
print("")
import time
start_time = time.time()

data_lemma = []
cpt = 0
for txt in data_c:
#     print(cpt, "/", len(data_c))
    cpt+=1
    lis = []
    doc = spacy_nlp(txt)
    data_lemma.append([token.lemma_ for token in doc if not (token.is_punct or token.is_stop) and len(token) > 3])
#     for token in doc:
# #         print(f"token de base : {token}, lematisé : {token.lemma_}, un stopword ? {token.is_stop}")
#         if not (token.is_punct or token.is_stop) and len(token) > 3:
# #             print(token, "->", token.lemma_)
#             lis.append(token.lemma_)
#     data_lemma.append(' '.join(lis))
# print(len(data_lemma))
print("--- %s seconds ---" % (time.time() - start_time))


--- 10.598894357681274 seconds ---


In [59]:
# équivalent à la cell du dessus
# 1 minutes -> (4469, 1768921)
import time
start_time = time.time()
data_lemma = [[token.lemma_ for token in spacy_nlp(txt) if not (token.is_punct or token.is_stop) and len(token) > 3] for txt in data_c]
print("--- %s seconds ---" % (time.time() - start_time))

--- 64.7949697971344 seconds ---


In [62]:
# Create Dictionary
dictionary = corpora.Dictionary(data_lemma)

# terme Doc Freqence
corpus = [dictionary.doc2bow(text) for text in data_lemma]


In [63]:
start_time = time.time()
##
## Multicore LDA
NUM_TOPICS = 3
lda_multicore_model = gensim.models.ldamulticore.LdaMulticore(corpus, num_topics = NUM_TOPICS, id2word=dictionary,random_state=100,passes=10)
# Saving trained model
lda_multicore_model.save('LDA_NYT_multicore')
# Loading trained model
lda_multicore_model = gensim.models.ldamodel.LdaModel.load('LDA_NYT_multicore')
## Print time taken to train the model
print("--- %s seconds ---" % (time.time() - start_time))

--- 14.618411302566528 seconds ---


In [64]:
# To plot at Jupyter notebook
pyLDAvis.enable_notebook()
plot = pyLDAvis.gensim.prepare(lda_multicore_model, corpus, dictionary)
# Save pyLDA plot as html file
pyLDAvis.save_html(plot, 'LDA_NYT.html')
plot

C:\Users\Utilisateur\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.021873 -0.056960       1        1  39.569725
0     -0.050195  0.043765       2        1  33.845341
2      0.072068  0.013195       3        1  26.584938, topic_info=     Category        Freq            Term        Total  loglift  logprob
term                                                                    
378   Default  909.000000         produit   909.000000  30.0000  30.0000
205   Default  780.000000         énergie   780.000000  29.0000  29.0000
210   Default  410.000000     agriculture   410.000000  28.0000  28.0000
211   Default  204.000000          animal   204.000000  27.0000  27.0000
227   Default  261.000000       pesticide   261.000000  26.0000  26.0000
218   Default  174.000000          espèce   174.000000  25.0000  25.0000
1304  Default  137.000000          chasse   137.000000  24.0000  24.0000
365   Default  526.000000         voiture   526.000000  23.0000  23.0000
702   Default  227.000000    renouvelable   227.000000  22.0000  22.0000
543   Default  328.000000       interdire   328.000000  21.0000  21.0000
212   Default  325.000000         arrêter   325.000000  20.0000  20.0000
21    Default  411.000000    consommation   411.000000  19.0000  19.0000
1829  Default  185.000000     énergétique   185.000000  18.0000  18.0000
303   Default  146.000000       Interdire   146.000000  17.0000  17.0000
93    Default  249.000000         changer   249.000000  16.0000  16.0000
1529  Default  144.000000  investissement   144.000000  15.0000  15.0000
1085  Default  100.000000        protéger   100.000000  14.0000  14.0000
11    Default  154.000000    biodiversité   154.000000  13.0000  13.0000
420   Default  452.000000           ville   452.000000  12.0000  12.0000
140   Default  157.000000         fossile   157.000000  11.0000  11.0000
635   Default  205.000000         solaire   205.000000  10.0000  10.0000
209   Default  229.000000     agriculteur   229.000000   9.0000   9.0000
247   Default  992.000000           faire   992.000000   8.0000   8.0000
665   Default  102.000000       isolation   102.000000   7.0000   7.0000
844   Default   62.000000            haie    62.000000   6.0000   6.0000
554   Default  158.000000          humain   158.000000   5.0000   5.0000
191   Default  219.000000         travail   219.000000   4.0000   4.0000
92    Default  187.000000        agricole   187.000000   3.0000   3.0000
201   Default  553.000000        véhicule   553.000000   2.0000   2.0000
190   Default  953.000000       transport   953.000000   1.0000   1.0000
...       ...         ...             ...          ...      ...      ...
1096   Topic3   42.039253        intensif    60.972771   0.9530  -6.6871
93     Topic3  138.476212         changer   249.975494   0.7342  -5.4950
382    Topic3   58.128204          viande    89.568382   0.8925  -6.3630
212    Topic3  169.318832         arrêter   325.589569   0.6710  -5.2939
1124   Topic3   73.068542        chimique   119.414520   0.8336  -6.1343
21     Topic3  200.735886    consommation   411.333740   0.6074  -5.1237
672    Topic3   53.906353         circuit    83.462006   0.8877  -6.4384
1510   Topic3   61.152187         culture    97.990166   0.8533  -6.3123
202    Topic3  122.905617            zone   243.573212   0.6408  -5.6142
659    Topic3   82.457207          nature   148.292847   0.7379  -6.0134
374    Topic3  144.396896           local   333.687500   0.4872  -5.4531
370    Topic3  173.983673       favoriser   442.966675   0.3903  -5.2667
420    Topic3  170.647308           ville   452.748291   0.3491  -5.2861
919    Topic3   70.686485          espace   125.048965   0.7544  -6.1674
350    Topic3  146.112427   environnement   367.641907   0.4021  -5.4413
95     Topic3   99.977608            mode   241.496216   0.4429  -5.8207
289    Topic3  138.576126      production   435.258972   0.1803  -5.4942
86     Topic3  209.305573         fal